In [ ]:
#inv_id = 1
inv_id = 'sum'
log_controller = 'testservice_log.csv'
log_prefix = '_testlog'

system = 'Windows_cyder'

system_dict = {
    'Windows_cyder':{
        'rootdir':r'D:\Users\emma\Documents\GitHub\CyDER\hil\scripts\ram',
        'storedir':r'D:\Users\emma\Documents\GitHub\CyDER\hil\scripts\sd',
        'optimization_root':r'D:\Users\emma\Documents\GitHub\SmartInverter\smartinverter_optimization',
        'fmlc_root':r'D:\Users\emma\Documents\GitHub\SmartInverter\fmlc',
        'control_root':r'D:\Users\emma\Documents\GitHub\SmartInverter\smartinverter_controller\RaspberryPi\TestService',
    }
} 

# Environment

In [ ]:
import os
import sys
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import traceback
import logging
#import matplotlib.pyplot as plt
#%matplotlib inline

logger = logging.getLogger(__name__)

# DOPER specific paths
rootdir = system_dict[system]['rootdir'] # Directory on RAM
storedir = system_dict[system]['storedir'] # Directory on SD-card
optimization_root = system_dict[system]['optimization_root'] # Directory of Optimizaiton
fmlc_root = system_dict[system]['fmlc_root'] # Directory of FMLC
control_root = system_dict[system]['control_root'] # Directory of Reaspberry root (should all be in FMLC)
sys.path.append(optimization_root)
sys.path.append(os.path.join(optimization_root, 'SlowActing'))
sys.path.append(os.path.join(optimization_root, 'Forecasting'))
sys.path.append(os.path.join(fmlc_root))
sys.path.append(os.path.join(control_root))

In [ ]:
# Import FMLC
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack

# Import Utility blocks (in framework folder)
from RealTimeClock import RealTimeClock
from InverterTelemetry import InverterTelemetry
from InverterControl import InverterControl

# Import Forecast functions (in smartinverter_optimization/Forecasting)
from ForecastMPC import mpc_input, warmstart_mpc_input

# Import Control functions (in smartinverter_optimization/SlowActing)
from SlowActingController import slowacting

# Exchange

In [ ]:
def exchange(configuration_file, simulationTime, input_names,
             input_values, output_names, write_results, memory):
    
    # Parameter
    flexgrid_host = 'http://131.243.41.47:9090' # Address to Flexgrid
    except_pass = False # Continue on error?
    
    simulationTime = time.time()
    
    log_dt = datetime.now().strftime('%Y%m%dT%H%M')+log_prefix
    if memory is None:
        memory = {'tLast':simulationTime, 'outputs':None}
        flowcontrol, controller = initialize_controller(simulationTime,
                                                        flexgrid_host=flexgrid_host,
                                                        control_flexgrid=False,
                                                        parallel=False,
                                                        debug=False,
                                                        log_dt=log_dt)
        flowcontrol, controller, error = evaluate_mpc(simulationTime, flowcontrol, controller,
                                                      except_pass=except_pass, log_dt=log_dt)
        memory['flowcontrol'] = flowcontrol
        memory['controller'] = controller
            
        pbatt = 0
        pload = 0
        error = 0
        memory['outputs'] = [pbatt, pload, error] 
    if (abs(simulationTime - memory['tLast'])>2):
        memory['tLast'] = simulationTime
        flowcontrol, controller, error = evaluate_mpc(simulationTime, memory['flowcontrol'],
                                                      memory['controller'],
                                                      except_pass=except_pass, log_dt=log_dt)
        memory['flowcontrol'] = flowcontrol
        memory['controller'] = controller
        
        # Print Telemetry
        telemetry = controller.get_output('telemetry', keys=['p_batt_meas','soc_batt_meas'])
        for k, v in telemetry.items():
            telemetry[k] = round(telemetry[k], 3)
        logger.info('Telemetry: {!s}'.format(telemetry))
        
        pbatt = float(controller.get_output('slowacting', keys=['p_batt'])['p_batt'])
        pload = controller.get_output('slowacting', keys=['Load Power [kW]'])['Load Power [kW]']
        pload = pload[min(pload.keys())] * 1000
        #print pbatt, pload

        if error == 1:
            memory['outputs'] = [0, 0, error]
        else:
            memory['outputs'] = [pbatt, pload, error]

    return [memory['outputs'], memory]

In [ ]:
def initialize_controller(simulationTime, flexgrid_host='', control_flexgrid=False,
                          parallel=False, debug=False, log_dt=''):
    # Setup logging
    logfile = os.path.join(rootdir, log_controller)
    if os.path.exists(logfile): os.remove(logfile)

    # Setup timing
    ts_ctrl = 60*5 # 5 minutes
    #ts_ctrl = 60*2 # 2 minutes

    ts = {} # seconds
    ts['main'] = 0.1
    ts['print'] = 60*60*999999
    ts['log'] = 60*5
    ts['store'] = 60*60*999999
    flowcontrol = triggering(ts)

    # Register and Initialize controller
    controller = {}
    controller['rtc_slowacting'] = {'fun':RealTimeClock, 'sampletime':ts_ctrl}
    controller['telemetry'] = {'fun':InverterTelemetry, 'sampletime':'rtc_slowacting'}
    controller['forecast'] = {'fun':mpc_input, 'sampletime':'telemetry'}
    controller['slowacting'] = {'fun':slowacting, 'sampletime':'forecast'}
    controller['ctrl_slowacting'] = {'fun':InverterControl, 'sampletime':'slowacting'}
    #controller['ctrl_refresh'] = {'fun':InverterControl, 'sampletime':60*30}
    tz = -8
    controller = controller_stack(controller, tz=tz, debug=debug, parallel=parallel)
    mapping = {}
    mapping['rtc_slowacting_time'] = -1
    mapping['forecast_time'] = 'rtc_slowacting_time'
    for k in ['generation_pv','load_demand']:
        mapping['slowacting_'+k] = 'forecast_'+k  
    mapping['telemetry_host'] = flexgrid_host
    mapping['telemetry_inv_id'] = inv_id
    mapping['slowacting_soc_initial'] = 'telemetry_soc_batt_meas'
    mapping['slowacting_soc_min'] = 0.1

    mapping['slowacting_soc_initial_prev'] = 'slowacting_soc_initial_used'
    mapping['slowacting_p_batt_inv'] = 'telemetry_p_batt_meas'
    mapping['slowacting_p_batt_prev'] = 'slowacting_p_batt'
    mapping['slowacting_ts'] = int(ts_ctrl/60.0)

    for k,v in {'p_lim':-1,'pf':-1,'host':flexgrid_host,'inv_id':inv_id,
                'control':int(control_flexgrid)}.iteritems():
        mapping['ctrl_slowacting_'+k] =  v 
        #mapping['ctrl_refresh_'+k] =  v  
    mapping['ctrl_slowacting_p_batt'] = 'slowacting_p_batt'   
    #mapping['ctrl_refresh_p_batt'] = 'ctrl_slowacting_p_batt'  
    #mapping['slowacting_soc_predicted_prev'] = 'slowacting_soc_predicted'
    mapping['forecast_forecast_pv_prev'] = 'forecast_forecast_pv_next'
    mapping['forecast_forecast_load_prev'] = 'forecast_forecast_load_next'


    # Fixed inputs
    mapping['forecast_location_latitude'] = 37.87
    mapping['forecast_location_longitude'] = -122.27
    from ComputeTariff import tariff_pge_e19
    mapping['slowacting_tariff'] =  tariff_pge_e19()
    mapping['slowacting_tz_st'] = tz
    mapping['forecast_tz_st'] = tz
    mapping['slowacting_tz_local'] = 'US/Pacific'
    path_forecasting = optimization_root

    models = {}
    #models['regression'] = {'history':2, 'prediction':0}
    models['regression'] = {}
    models['sarima'] = {'path':os.path.join(optimization_root,'Forecasting',
                                            'models/all/20181029/SARIMA_model_20181025.json'),
                        'normPowerCoeff': 2626.0}
    '''
    # No NN because it is not retrained 
    models['nn'] = {'path':os.path.join(optimization_root,'Forecasting',
                                        'models/all/20181029/NNmodel_best_20181025.sav'),
                    'normPowerCoeff': 2626.0, 'normInputData':True,
                    'normTa': 30.0, 'normCC': 100.0, 'normCS': 1000.0,
                    'architecture':'scalar',
    #                            temp  cloud sky   Pd-1  hor                    
                    'inputData':[True, True, True, True, True],
                    'randomSeed': 10}
    '''
    models['nn'] = {}
    models['alpha'] = {'path':os.path.join(optimization_root,'Forecasting',
                                           'models/all/20181029/optimal_weighting_factors_20181025.json')}
    mapping['forecast_models'] =  models


    #mapping['forecast_config'] = {'pv_norm':11682}
    mapping['forecast_config'] = {'scale_pv':1, 'scale_load':15/5.4} # Scale load to 15 kW (same as PV)
    mapping['forecast_external_weather_forecast'] = False

    # Warm start
    p_pv, p_pv_hist = warmstart_mpc_input(simulationTime, inv_id=inv_id, tz=tz)
    mapping['forecast_p_pv'] = 'telemetry_p_pv_meas'
    mapping['forecast_p_pv_hist'] = 'forecast_p_pv_hist_update'
    mapping['forecast_p_pv_hist_init'] = p_pv_hist
    mapping['forecast_wf_prev'] = 'forecast_weather_forecast'

    # Initialize controller
    controller.initialize(mapping, now = simulationTime)

    print controller.database.address
    #n = 3
    #fig, axs = plt.subplots(n,1, figsize=(12, n*3), sharex=True, sharey=False)
    #axs = axs.ravel()

    # Start Control Loop
    end = simulationTime + 60*60*24*5

    config_live = {}
    config_live['db'] = controller.database.address
    config_live['log_ram'] = log_dt
    with open('config_live.json', 'wb') as f:
        json.dump(config_live, f)
        
    return flowcontrol, controller

In [ ]:
def evaluate_mpc(simulationTime, flowcontrol, controller, except_pass=False, log_dt=''):
    error = 0
    now = simulationTime
    if now >= flowcontrol.trigger['main']:
        try:
            controller.query_control(now) # Check if controller due to compute

            if now >= flowcontrol.trigger['print']: # Print logs
                print datetime.now().replace(microsecond=0),
                print controller.get_input('slowacting', keys=['soc_initial']),
                print controller.get_input('ctrl_slowacting', keys=['p_batt','control'])
                #axs[0].plot(utility_mpc_to_df(controller,name='slowacting')['Battery Power [kW]'])
                #axs[1].plot(utility_mpc_to_df(controller,name='slowacting')['Battery SOC [-]'])
                #axs[2].plot(utility_mpc_to_df(controller,name='slowacting')['Load Power [kW]'])
                flowcontrol.refresh_trigger('print', now)

            if now >= flowcontrol.trigger['log']: # Write log files to ram drive
                print datetime.now().replace(microsecond=0), 'Log to RAM'
                controller.log_to_csv(path=os.path.join(rootdir, log_dt))
                flowcontrol.refresh_trigger('log', now)

            if now >= flowcontrol.trigger['store']: # Copy file from ram to sd card
                print datetime.now().replace(microsecond=0), 'Log to SD'                
                sp.call('mv {}/* {}'.format(rootdir, storedir), shell=True)
                controller.clear_logs()
                flowcontrol.refresh_trigger('store', now)
                log_dt = (pd.to_datetime(flowcontrol.trigger['main'], unit='s').replace(microsecond=0, nanosecond=0)+ \
                          pd.DateOffset(hours=tz)).to_pydatetime()
                log_dt = log_dt.strftime('%Y%m%dT%H%M')+log_prefix

        except Exception as e:
            print ('***ERROR***')
            print (flowcontrol.trigger['main'], datetime.now())
            print (traceback.print_exc())
            print (e)
            if except_pass: pass
            else: raise Exception('Exiting because of an Error (except_pass == False)')
            error = 1

        flowcontrol.refresh_trigger('main', now)
        
    return flowcontrol, controller, error

In [ ]:
if __name__ == '__main__':
    memory = None
    for i in range(4):
        now = time.time()
        res, memory = exchange(None, now, None, None, None, False, memory)
        print res
        time.sleep(5)